In [1]:
import sys
sys.path.append(r'C:\Users\kuina\OneDrive\TFG\Codigo')

In [2]:
from datasets import load_dataset
import json

from utils_creacion_datasets import create_dependency_tags

from utils_generic import text_to_num,num_to_text

## Datos Huggingface

## TRAIN

In [24]:
convai2_about = load_dataset('md_gender_bias','convai2_inferred')
train_about = convai2_about['train']
print(train_about[0])

Reusing dataset md_gender_bias (C:\Users\kuina\.cache\huggingface\datasets\md_gender_bias\convai2_inferred\1.0.0\8ae77b51acf93383161cc954b146159291beca6c979b54ce228c46db86116c05)


  0%|          | 0/3 [00:00<?, ?it/s]

{'text': "hi , how are you doing ? i'm getting ready to do some cheetah chasing to stay in shape .", 'binary_label': 1, 'binary_score': 0.6521999835968018, 'ternary_label': 2, 'ternary_score': 0.4496000111103058}


In [25]:
train_about = train_about.remove_columns(['binary_score', 'ternary_label', 'ternary_score'])
print(train_about)

Dataset({
    features: ['text', 'binary_label'],
    num_rows: 131438
})


In [26]:
#Filtro repetidos para que sean una única entrada (Aqui aun no estoy filtrando las etiquetas)
dataT = {}
print(f'Elementos antes de filtrar repetidos = {len(train_about)}')
for item in train_about:
    text = item['text']
    label = item['binary_label']
    if text not in dataT:
        # data[text] =[label]
        dataT[text] =[num_to_text['about'][label]]
    else:
        dataT[text].append(num_to_text['about'][label])

print(f'Elementos después de filtrar repetidos = {len(dataT)}')

# Filtro las etiquetas para que no haya items repetidos

for item in dataT:
    dataT[item] = list(set(dataT[item]))


Elementos antes de filtrar repetidos = 131438
Elementos después de filtrar repetidos = 115877


## VAL

In [27]:

val_about = convai2_about['validation']
print(val_about[0])
val_about = val_about.remove_columns(['binary_score', 'ternary_label', 'ternary_score'])
print(val_about)

{'text': 'hello what are doing today ?', 'binary_label': 1, 'binary_score': 0.5015000104904175, 'ternary_label': 1, 'ternary_score': 0.3422999978065491}
Dataset({
    features: ['text', 'binary_label'],
    num_rows: 7801
})


In [33]:
#Filtro repetidos para que sean una única entrada (Aqui aun no estoy filtrando las etiquetas)
dataV = {}
print(f'Elementos antes de filtrar repetidos = {len(val_about)}')
for item in val_about:
    text = item['text']
    label = item['binary_label']
    if text not in dataV:
        # data[text] =[label]
        dataV[text] =[num_to_text['about'][label]]
    else:
        dataV[text].append(num_to_text['about'][label])

print(f'Elementos después de filtrar repetidos = {len(dataV)}')

# Filtro las etiquetas para que no haya items repetidos

for item in dataV:
    dataV[item] = list(set(dataV[item]))

Elementos antes de filtrar repetidos = 7801
Elementos después de filtrar repetidos = 7503


## Carga datos ParlAI

## TRAIN

In [28]:
with open(r'./convai2_parlai_train.json') as f: #Este tiene unks
    convai2_parlai_train = json.load(f)

## VAL

In [13]:
with open(r'./convai2_parlai_val.json') as f: #Este tiene unks
    convai2_parlai_val = json.load(f)

# Unión de los datasets

## TRAIN

Me interesa tener 3 etiquetas, por lo que quiero las que oraciones que salgan en convai2_parlai y en data (etiquetas about the convai2). Itero sobre convai2_parlai porque es el conjunto más pequeño de los 2

In [29]:
join_data = {}
for item in convai2_parlai_train:
    if item in dataT:
        join_data[item] = {'label_about':dataT[item],
                            'label_to':convai2_parlai_train[item]['partner'],
                            'label_as':convai2_parlai_train[item]['self']}


In [30]:
len(join_data)

73398

In [31]:
with open('convai2_complete_train.json','w') as f:
    json.dump(join_data,f)

## VAL

In [35]:
join_data = {}
for item in convai2_parlai_val:
    if item in dataV:
        join_data[item] = {'label_about':dataV[item],
                            'label_to':convai2_parlai_val[item]['partner'],
                            'label_as':convai2_parlai_val[item]['self']}


In [36]:
len(join_data)

7503

In [37]:
with open('convai2_complete_val.json','w') as f:
    json.dump(join_data,f)